<a href="https://colab.research.google.com/github/kartikvemulapalli/project/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from glob import glob
# Import the necessary patch to display images in Colab
from google.colab.patches import cv2_imshow

# Function to extract color histogram features from an image
def extract_features(image_path, bins=(8, 8, 8)):
    image = cv2.imread(image_path)
    if image is None: # Check if the image was loaded successfully
        print(f"Error: Could not load image at {image_path}")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

# Load and extract features for all images in the database
image_paths = glob('/content/images/*.jpg')
features = []
for path in image_paths:
    features.append(extract_features(path))

# Convert the features to a NumPy array for easier manipulation
features = np.array([f for f in features if f is not None]) # Remove any None values

# Query image (the image to compare against the others)
query_image_path = "/content/images/cdmc953.jpg" # Make sure this path is correct
query_features = extract_features(query_image_path)

if query_features is not None: # Proceed only if the query image was loaded
    # Compute similarity between the query image and all images in the database
    similarities = cosine_similarity([query_features], features)[0]

    # Find the most similar images (e.g., top 5)
    top_indices = np.argsort(similarities)[::-1][:5]  # Top 5 most similar images
# Display the top results
    for i in top_indices:
        print(f"Image: {image_paths[i]}, Similarity: {similarities[i]}")
        img = cv2.imread(image_paths[i])
        # Use cv2_imshow instead of cv2.imshow
        cv2_imshow(img)

    cv2.destroyAllWindows()